# Goal: to use fabric to setup entire workflow on Raspberry pi
inspired by https://www.youtube.com/watch?v=mzP-QYxW9Vs&list=PL2hxivbr6C_2lMqtZi4uVAOta2b5dp1yN&index=28&t=0s&ab_channel=DavyWybiral

3. Enable SSH on a headless Raspberry Pi (add file to SD card on another machine)
For headless setup, SSH can be enabled by placing a file named ssh, without any extension, onto the boot partition of the SD card from another computer. When the Pi boots, it looks for the ssh file. If it is found, SSH is enabled and the file is deleted. The content of the file does not matter; it could contain text, or nothing at all.

If you have loaded Raspberry Pi OS onto a blank SD card, you will have two partitions. The first one, which is the smaller one, is the boot partition. Place the file into this one.

In [21]:
# !pip uninstall fabric

In [26]:
# !pip install fabric2

In [9]:
# !pip install python-dotenv

In [10]:
from dotenv import load_dotenv
import os

In [138]:
load_dotenv()

True

In [139]:
pwd = os.getenv('PI_PSWD')
pi1 = os.getenv('PI_IP1')
pi_uid = os.getenv('PI_UID')
pi_gid = os.getenv('PI_GID')

# Using Fabric

In [115]:
from fabric import Connection, Config

In [188]:
config = Config({'user': 'pi', 'connect_kwargs': {'password': pwd}})

In [229]:
cxn = Connection(pi1, config = config)

In [226]:
cxn.run('whoami')

pi


<Result cmd='whoami' exited=0>

In [211]:
def reboot():
    # reboot hosts
    cxn.sudo('shutdown -r now')


def shutdown():
    # shutdown hosts
    cxn.sudo('shutdown -h now')


def update():
    # apt update & apt dist-upgrade hosts
    cxn.sudo('apt-get update')
    cxn.sudo('apt-get dist-upgrade -y') 

def change_uid_gid():
    #TODO create a new user and change the pi user's UID GID
    cxn.sudo(f'usermod -u {pi_uid} pi')
    cxn.sudo(f'groupmod -g {pi_gid} pi')
    #Or change below files (since processes with current PID is running)
#   sudo nano  /etc/passwd
#   sudo nano  /etc/group
#   while we change the pi's PID in passwd, we need to ssh again into the terminal to use
#   sudo to change the secound group file once done, we need to restart pi using
#   sudo su to have root user access then
#   systemctl reboot -i
    
    
def linux_setup():
    for i in ['apt-get install -y python3-pip tree htop nfs-common',
    'rm -rf /var/lib/apt/lists/*']:
        cxn.sudo(i)
        
def setup_nas():
    nas_ip = os.getenv('NAS_IP')
    cxn.sudo('mkdir -p /home/pi/pi_projects')
    mount_to_pi = f'mount {nas_ip}:/volume1/pi_projects /home/pi'
    cxn.sudo(mount_to_pi)
    
def add_mount_nas_to_restart():
    # add to restart
    cxn.sudo("chmod 777 /etc/fstab")
    cxn.sudo(f"echo '{nas_ip}:/volume1/pi_projects /home/pi nfs rsize=8192,wsize=8192,timeo=14,intr'  >> /etc/fstab ")
    cxn.run('tail /etc/fstab')
    
def docker_setup():
    cxn.sudo('apt-get install apt-transport-https ca-certificates software-properties-common -y')
    cxn.sudo('curl -fsSL get.docker.com -o get-docker.sh && sh get-docker.sh')
    cxn.sudo('usermod -aG docker pi')
    cxn.sudo('curl https://download.docker.com/linux/raspbian/gpg')
    cxn.sudo("chmod 777 /etc/apt/sources.list")
    cxn.sudo("""echo "deb https://download.docker.com/linux/raspbian/ stretch stable" >> /etc/apt/sources.list""")
    cxn.run('tail /etc/apt/sources.list')
    cxn.sudo('pip3 install docker-compose')
    
def docker_start():
    cxn.sudo('systemctl start docker.service')
    cxn.run('docker info')

def docker_file_upload():
    cxn.run('cd /home/pi/Docker')
    cxn.put('Dockerfile', '/home/pi/Docker')
    cxn.put('requirements.txt', '/home/pi/Docker')
    cxn.put('docker-compose.yml', '/home/pi/Docker')
    
    #======================
# @parallel
def rq_setup():
    # setup hosts (install dependencies)
    cxn.sudo('pip3 install rq redis')

# @parallel
def upload():
    # upload code to hosts
    cxn.put('tasks.py', 'tasks.py')
    cxn.put('worker.py', 'worker.py')

# @parallel
def start():
    # start worker.py scripts
    cxn.run('python3 worker.py')

In [202]:
# reboot()

In [195]:
# update()
# linux_setup()
# setup_nas()
# docker_setup()
# docker_start()
# rq_setup()

In [203]:
# add_mount_nas_to_restart()

In [230]:
docker_file_upload()

In [231]:
cxn.run('tree /home/pi')

/home/pi
├── Docker
│   ├── docker-compose.yml
│   ├── Dockerfile
│   └── requirements.txt
├── get-docker.sh
├── #recycle [error opening dir]
└── test

3 directories, 4 files


<Result cmd='tree /home/pi' exited=0>